In [17]:
%matplotlib inline

In [18]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm import tqdm

torch.manual_seed(17)
np.random.seed(17)
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
PATH = './resnet_net'
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu = torch.device("cpu")

PyTorch Version:  2.0.0+cu117
Torchvision Version:  0.15.1+cu117


In [19]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 8

# Batch size for training (change depending on how much memory you have)
batch_size = 16

# Number of epochs to train for 
num_epochs = 5

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = True

In [20]:
# INPUTS: output have shape of [batch_size, category_count]
#    and target in the shape of [batch_size] * there is only one true class for each sample
# topk is tuple of classes to be included in the precision
# topk have to a tuple so if you are giving one number, do not forget the comma
def accuracy_pk(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
   #we do not need gradient calculation for those
    with torch.no_grad():
    #we will use biggest k, and calculate all precisions from 0 to k
        maxk = max(topk)
        batch_size = target.size(0)
    #topk gives biggest maxk values on dimth dimension from output
    #output was [batch_size, category_count], dim=1 so we will select biggest category scores for each batch
    # input=maxk, so we will select maxk number of classes 
    #so result will be [batch_size,maxk]
    #topk returns a tuple (values, indexes) of results
    # we only need indexes(pred)
        _, pred = output.topk(input=maxk, dim=1, largest=True, sorted=True)
    # then we transpose pred to be in shape of [maxk, batch_size]
        pred = pred.t()
   #we flatten target and then expand target to be like pred 
   # target [batch_size] becomes [1,batch_size]
   # target [1,batch_size] expands to be [maxk, batch_size] by repeating same correct class answer maxk times. 
   # when you compare pred (indexes) with expanded target, you get 'correct' matrix in the shape of  [maxk, batch_size] filled with 1 and 0 for correct and wrong class assignments
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        """ correct=([[0, 0, 1,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8) """
        res = []
       # then we look for each k summing 1s in the correct matrix for first k element.
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


In [21]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [22]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


transform = transforms.Compose([
    transforms.RandomResizedCrop(input_size),
    transforms.RandomHorizontalFlip(),    
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    ])

print("Initializing Datasets and Dataloaders...")


import os
from PIL import Image
school_labels = ["zxb","nm","sy","tsg","mhb","sjz","tyht"]
other_labels = ["all_souls","ashmolean","balliol","bodleian","christ_church","cornmarket","hertford","jesus","keble","magdalen","new","oriel","oxford","pitt_rivers","radcliffe_camera","trinity","worcester"]
all_labels = school_labels + ["other"] #other_labels
print(len(all_labels))

label_dict = {}
for i, tag in enumerate(all_labels):
    label_dict[tag] = i


class BJTUDataset(torch.utils.data.Dataset):
    def __init__(self, root, train=True, transform=None):
        self.root = root
        self.transform = transform
        self.train = train
        self.image_files = os.listdir(root)
        self.image_paths = [os.path.join(root, imgf) for imgf in self.image_files]
        # self.images = [Image.open(image_path) for image_path in self.image_paths]
        # self.images = []
        # cot=0
        # for image_path in self.image_paths:
            

        #     try:
        #         cot+=1
        #         self.images.append(Image.open(image_path))
        #         print("{} images loaded".format(cot))
        #     except Exception as err:
        #         print(err)
        #         print("Invalid image: {}".format(image_path))
        #         self.image_paths.remove(image_path)
        
        self.prelabels = [img.split('-')[0] for img in self.image_files]
        for i, label in enumerate(self.prelabels):
            if label in school_labels:
                self.prelabels[i] = label
            else:
                self.prelabels[i] = "other"
                
        self.labels = [label_dict[label] for label in self.prelabels]
        # self.N = len(self.image_files)
        #print first 10 labels
        print(self.labels[:10])
        #print first 10 image paths
        print(self.image_paths[:10])


    def __len__(self):
        # print("Total images: {}".format(self.N))
        return len(self.image_paths)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = Image.open(self.image_paths[idx])
        # if image.mode != 'RGB':
        #     image=image.convert('RGB')
        # print("Reading image: {}".format(self.image_paths[idx]))
        # image = self.images[idx]
        # self.transform = False
        # image = np.array(image)
        if self.transform:
            image = self.transform(image)
        return image, label

work_path = "C:\\Left\\Workspace\\BJTU-CVFundamental\\"
data_root = os.path.join(work_path, "data\\bjtu-dataset-mixed\\Cleaned_Dataset_1024p\\")

trainset = BJTUDataset(root=data_root+'train', train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = BJTUDataset(root=data_root+'test', train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

dataloaders_dict = {'train':trainloader, 'val':testloader}

Initializing Datasets and Dataloaders...
8
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
['C:\\Left\\Workspace\\BJTU-CVFundamental\\data\\bjtu-dataset-mixed\\Cleaned_Dataset_1024p\\train\\all_souls-0135678249.jpg', 'C:\\Left\\Workspace\\BJTU-CVFundamental\\data\\bjtu-dataset-mixed\\Cleaned_Dataset_1024p\\train\\all_souls-0172463589.jpg', 'C:\\Left\\Workspace\\BJTU-CVFundamental\\data\\bjtu-dataset-mixed\\Cleaned_Dataset_1024p\\train\\all_souls-0185649723.jpg', 'C:\\Left\\Workspace\\BJTU-CVFundamental\\data\\bjtu-dataset-mixed\\Cleaned_Dataset_1024p\\train\\all_souls-0238156479.jpg', 'C:\\Left\\Workspace\\BJTU-CVFundamental\\data\\bjtu-dataset-mixed\\Cleaned_Dataset_1024p\\train\\all_souls-0273958641.jpg', 'C:\\Left\\Workspace\\BJTU-CVFundamental\\data\\bjtu-dataset-mixed\\Cleaned_Dataset_1024p\\train\\all_souls-0278913465.jpg', 'C:\\Left\\Workspace\\BJTU-CVFundamental\\data\\bjtu-dataset-mixed\\Cleaned_Dataset_1024p\\train\\all_souls-0346295718.jpg', 'C:\\Left\\Workspace\\BJTU-CVFundamental\\data\\bj

In [24]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


def eval_model(model, eval_dataloader, criterion):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    model.eval()   # Set model to evaluate mode

    running_loss = 0.0
    running_corrects = 0

    topKsamples = 60

    
    all_outputs = [[]] * len(all_labels)
    # Iterate over data.
    for inputs, labels in tqdm(eval_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels_copy = labels.clone().to(cpu).numpy()
        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            
            outputs = model(inputs)
            
            # print(outputs.shape)
            #append outputs to all_outputs
            for i in range(len(labels)):
                all_outputs[labels_copy[i]].append((outputs[i].clone().to(cpu).numpy() , labels_copy[i]))

            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)


    # print(np.array(all_outputs).shape)

    #sort all_outputs
    labels_precision = []

    labels_line = []

    for i in range(len(all_outputs)):
        labels_line.append([])
        #P@K Pn Kn
        labels_line[i].append((topKsamples,0))
        precision_label = 0
        all_outputs[i].sort(key=lambda x: x[0][i], reverse=True)
        all_outputs[i] = all_outputs[i][:topKsamples]

        # print(all_outputs[i])
        for j in range(len(all_outputs[i])):
            all_outputs[i][j]=(np.argmax(all_outputs[i][j][0]),i)
        
        # print(all_outputs[i])
        for j in range(len(all_outputs[i])):
            L=len(labels_line[i])
            Last = labels_line[i][-1]
            if all_outputs[i][j][0] == all_outputs[i][j][1]:
                precision_label += 1
                labels_line[i].append( [ Last[0], Last[1]+1 ] )
            else:
                labels_line[i].append( [ Last[0]-1, Last[1] ] )
        precision_label /= len(all_outputs[i])
        labels_precision.append(precision_label)
        print('Precision on label[{}:{}]:{:.5f} '.format(all_labels[i],i,precision_label))

        np.save('labels_line.npy', np.array(labels_line))

    # accuracypk = accuracy_pk(outputs, labels, topk=(1, 5))

    epoch_loss = running_loss / len(eval_dataloader.dataset)
    epoch_acc = running_corrects.double() / len(eval_dataloader.dataset)

    print('{} Loss: {:.4f} Acc: {:.4f}'.format('val', epoch_loss, epoch_acc))
    # print('AccuracyPK: ', accuracypk)
    print()

    time_elapsed = time.time() - since
    print('Evaluation complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(epoch_acc))

    return model, epoch_acc, labels_precision

In [25]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [26]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# # Train and evaluate
# model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))


In [27]:
# eval_model(model_ft, testloader, criterion)

In [28]:

# torch.save(model_ft.state_dict(), PATH+("_{}.pth".format(time.time())))

In [29]:
class Subset(torch.utils.data.Dataset):
    """
    Subset of a dataset at specified indices.
    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
    """
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __len__(self):
        if self.indices.shape == ():
            print('this happens: Subset')
            return 1
        else:
            return len(self.indices)

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

k=7

num_val_samples = int(len(trainset) / k)
dataset=trainset

print('len(dataset): ', len(dataset))
print('num_val_samples: ', num_val_samples)

arr=np.arange(len(dataset))
np.random.shuffle(arr)
all_pk = []
if(False): #switch true to train

    for i in range(k):
        print('Processing fold: %d/%d' % (i + 1, k))
        """%%%% Initiate new model %%%%""" #in every fold
        
        valid_idx = arr[i * num_val_samples:(i + 1) * num_val_samples]
        train_idx = np.concatenate([arr[:i * num_val_samples], arr[(i + 1) * num_val_samples:]], axis=0)
        train_dataset = Subset(dataset, train_idx)
        valid_dataset = Subset(dataset, valid_idx)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
        # 
        # Create training and validation dataloaders
        dataloaders_dictk = {'train': train_loader, 'val': valid_loader}

        # Setup the loss fxn
        criterion = nn.CrossEntropyLoss()

        # Train and evaluate
        model_ft, hist = train_model(model_ft, dataloaders_dictk, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))
        
        model_ft, epoch_acc, pk = eval_model(model_ft, testloader, criterion)
        all_pk.append(pk)

    #cal average pk
    all_pk = np.array(all_pk)
    print(all_pk.shape)
    all_pk = np.mean(all_pk, axis=1)
    for i in range(len(all_pk)):
        print('Average Precision on label[{}:{}]:{:.5f} '.format(all_labels[i],i,all_pk[i]))
    
torch.save(model_ft.state_dict(), PATH+("_{}.pth".format(time.time())))

len(dataset):  7296
num_val_samples:  1042


In [30]:
PATH="FTresnet.pth"

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)

model_ft=model_ft.to(device)

model_ft.load_state_dict(torch.load(PATH))

eval_model(model_ft, testloader, criterion)

100%|██████████| 35/35 [00:09<00:00,  3.50it/s]

Precision on label[zxb:0]:0.58333 
Precision on label[nm:1]:1.00000 
Precision on label[sy:2]:0.98333 
Precision on label[tsg:3]:1.00000 
Precision on label[mhb:4]:0.48333 
Precision on label[sjz:5]:1.00000 
Precision on label[tyht:6]:1.00000 
Precision on label[other:7]:0.35000 
val Loss: 0.3021 Acc: 0.9011

Evaluation complete in 0m 10s
Best val Acc: 0.901079


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU